In [1]:
#| default_exp occlusion

In [1]:
#| export
import numpy as np

In [2]:
#| export
def apply_patch(image, top_left_x, top_left_y, patch_size):
    """
    Zeros-out a square region within the image
    
    input :
      - image : ndarray (Image to be occluded)
      - top_left_x : int (Top coordinate of the patch)
      - top_left_y : int (Left coordinate of the patch)
      - patch_size : int (Size of the square patch)
      
    return :
      - patched_image : ndarray (Occluded image)
    """
    patched_image = np.copy(image)
    patched_image[top_left_y : top_left_y+patch_size,
                  top_left_x : top_left_x+patch_size, :] = 0
    return patched_image

In [6]:
#| export
def generate_img_occlusion_list(img, patch_size, saliency=None, saliency_reduce_func=np.mean):
    """
    Generates a list of img that has been sequentially occluded according to the patch_size,
    whereby `stride=patch_size`.
    
    input :
      - img : ndarray (Image to create occluded version of)
      - patch_size : int (Size of the occluded patch)
      - saliency : ndarray (optional saliency map to generate a reduced saliency value for each occluded region)
      - saliency_reduce_func : function (optional iterative-wise function to reduce saliency value into a value)
      
    return :
      - occlusion_img_list : ndarray (Array of occluded versions of the image)
      - occlusion_saliency_statistic : ndarray|None (Array of reduce saliency value for each patch location)
    """
    occlusion_img_list = []
    occlusion_saliency_statistic = []
    if saliency is not None:
        saliency = np.array(Image.fromarray(saliency).resize((img.shape[0], img.shape[1])))
    
    for top_left_y in range(0, img.shape[1], patch_size):
        for top_left_x in range(0, img.shape[0], patch_size):
            patched_image = apply_patch(img, top_left_x, top_left_y, patch_size).astype('float32') / 255.0
            occlusion_img_list.append(patched_image)
            if saliency is not None:
                occlusion_saliency_statistic.append(saliency_reduce_func(saliency[top_left_y : top_left_y+patch_size,
                                                                                  top_left_x : top_left_x+patch_size]))
            
    return np.array(occlusion_img_list), (np.array(occlusion_saliency_statistic) if saliency is not None else None)

In [ ]:
#| export
def get_nonintersecting_patch_coord_mask(height, width, num_patch, patch_size):
    """
    Returns the coordinate of occlusion patches, as well as a binary mask where patch location have been zero'd out.
    The returned patch locations are such that they will not intersect with each other under any circumstance.
    Due to the aformentioned rule, if not enough space available, the function might return smaller number of
    patches than requested.
    
    input :
      - height : int (height of the desired mask)
      - width : int (width of the desired mask)
      - num_patch : int (maximum number of patches present in one occluded image)
      - patch_size : int (height and width of the patch)
      
    return :
      - patch_coords : ndarray (Array containing the 2D coordinates for the top-left location of patches)
      - mask_np : ndarray (A bianry mask that has been generated using the coodinates)
    """
    mask_np = np.ones((height, width))
    allowed_mask_pos = np.ones((height, width))
    allowed_mask_pos[width-patch_size:, :] = 0
    allowed_mask_pos[:, height-patch_size:] = 0
    
    sample_x = np.random.choice(np.where(np.any(allowed_mask_pos, 1, where=1))[0])
    sample_y = np.random.choice(np.where(allowed_mask_pos[sample_x,] != 0)[0])
    
    allowed_mask_pos[np.clip(sample_x-patch_size+1, 0, sample_x):sample_x+patch_size,
                     np.clip(sample_y-patch_size+1, 0, sample_y):sample_y+patch_size] = 0
    
    mask_np[sample_x:sample_x+patch_size, sample_y:sample_y+patch_size] = 0
    patch_coords = np.array([[sample_x, sample_y]])
    for _ in range(num_patch-1):
        remain_x = np.where(np.any(allowed_mask_pos, 1, where=1))[0]
        if remain_x.shape[0] == 0:
            break
            
        sample_x = np.random.choice(remain_x)
        sample_y = np.random.choice(np.where(allowed_mask_pos[sample_x,] != 0)[0])
        allowed_mask_pos[np.clip(sample_x-patch_size+1, 0, sample_x):sample_x+patch_size,
                         np.clip(sample_y-patch_size+1, 0, sample_y):sample_y+patch_size] = 0
        
        mask_np[sample_x:sample_x+patch_size, sample_y:sample_y+patch_size] = 0
        patch_coords = np.vstack((patch_coords, np.array([sample_x, sample_y])))
        
    return patch_coords, mask_np

In [ ]:
#| export
def generate_multipatch_occlusion_list_reduce_saliency(img : np.ndarray, patch_size : int, multipatch_num : int, steps : int, saliency=None, saliency_reduce_func=np.sum):
    """
    Generates a list of images that has been occluded on multiple locations, defined by multipatch_num,
    whereby the location of each patch is random and the patches never overlap.
    
    input :
      - img : ndarray (Image to create occluded version of)
      - patch_size : int (Size of the occluded patch)
      - multipatch_num : int (The maximum number of patches in the occluded image)
      - steps : int (Total number of occluded image)
      - saliency : ndarray (optional saliency map to generate a reduced saliency value for each occluded version of the image)
      - saliency_reduce_func : function (optional iterative-wise function to reduce saliency values into a value)
      
    return :
      - occlusion_img_list : ndarray (Array of occluded versions of the image)
      - occlusion_saliency_statistic : ndarray|None (Array of reduce saliency value for each patch location)
    """
    occlusion_img_list = []
    occlusion_saliency_statistic = []
    if saliency is not None:
        occlusion_saliency_statistic = []
        saliency = np.array(Image.fromarray(saliency).resize((img.shape[0], img.shape[1])))
    
    patch_locs = np.random.randint(0, img.shape[0]-patch_size, (steps, multipatch_num, 2))
    for step_idx in range(steps):
        # saliency_value = 
        patch_coords, mask = get_patch_coord_mask(img.shape[0], img.shape[1], multipatch_num, patch_size)
        patched_image = img * np.expand_dims(mask, -1)
        patched_image = patched_image.astype('float32') / 255.0
        occlusion_img_list.append(patched_image)
        if saliency is not None:
            occlusion_saliency_statistic.append(saliency_reduce_func(saliency * np.logical_not(mask)))
        
    return np.array(occlusion_img_list), (np.array(occlusion_saliency_statistic) if saliency is not None else None)

In [3]:
#| hide
from nbdev.export import nb_export
nb_export('occlusion.ipynb', '../../commons/api/method')